# Pig Detection CNN Project Template

This notebook scaffolds the workflow for the HW1 dense pig detection assignment. Replace each `### TODO ###` with your implementation.

## Project Setup

In [ ]:
import os
import json
import math
import random
from pathlib import Path

import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

# ### TODO ###: add any additional third-party libraries (albumentations, matplotlib, etc.)


In [ ]:
# ### TODO ###: adjust experiment metadata
EXPERIMENT_NAME = "pig_detection_baseline"
OUTPUT_DIR = Path("artifacts") / EXPERIMENT_NAME
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# ### TODO ###: configure dataset root and annotation paths
DATA_ROOT = Path("../HW1/data")
ANNOTATIONS_PATH = DATA_ROOT / "annotations.json"
IMAGES_DIR = DATA_ROOT / "images"

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")


In [ ]:
def set_seed(seed: int = 42) -> None:
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = False
    torch.backends.cudnn.benchmark = True

# ### TODO ###: choose an appropriate seed value
set_seed(42)


## Data Pipeline

In [ ]:
class PigDetectionDataset(Dataset):
    def __init__(self, annotations, images_dir: Path, transforms=None):
        self.annotations = annotations
        self.images_dir = Path(images_dir)
        self.transforms = transforms

    @classmethod
    def from_json(cls, annotations_path: Path, images_dir: Path, transforms=None):
        # ### TODO ###: parse the annotation format used in the assignment
        with open(annotations_path, "r") as fp:
            annotations = json.load(fp)
        return cls(annotations, images_dir, transforms)

    def __len__(self) -> int:
        # ### TODO ###: return dataset size
        return len(self.annotations)

    def __getitem__(self, idx: int):
        record = self.annotations[idx]
        # ### TODO ###: load image and target (bounding boxes, labels, etc.)
        image_path = self.images_dir / record.get("file_name", "")
        # ### TODO ###: replace placeholder with actual image loading logic
        image = None
        target = {}

        if self.transforms is not None:
            image, target = self.transforms(image, target)

        return image, target

# ### TODO ###: build any required preprocessing / augmentation transforms
train_transforms = None
val_transforms = None


In [ ]:
def create_dataloaders(batch_size: int = 4, num_workers: int = 4):
    # ### TODO ###: decide how to split annotations into train/val/test
    dataset = PigDetectionDataset.from_json(ANNOTATIONS_PATH, IMAGES_DIR, transforms=train_transforms)

    # ### TODO ###: implement a proper split instead of reusing the entire dataset
    train_dataset = dataset
    val_dataset = dataset

    def collate_fn(batch):
        return tuple(zip(*batch))

    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        collate_fn=collate_fn,
        pin_memory=True,
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        collate_fn=collate_fn,
        pin_memory=True,
    )

    return train_loader, val_loader

# ### TODO ###: set batch size and worker count
train_loader, val_loader = create_dataloaders(batch_size=4, num_workers=2)


## Model Definition

In [ ]:
class PigDetectionModel(nn.Module):
    def __init__(self):
        super().__init__()
        # ### TODO ###: define the CNN backbone and detection heads
        self.feature_extractor = nn.Identity()
        self.head = nn.Identity()

    def forward(self, images, targets=None):
        # ### TODO ###: implement forward pass and return outputs compatible with loss
        return {}

def build_model() -> PigDetectionModel:
    model = PigDetectionModel()
    return model.to(DEVICE)

model = build_model()
print(model)


In [ ]:
# ### TODO ###: define your loss function(s) and metrics
def compute_loss(outputs, targets):
    # Placeholder example loss
    loss = torch.tensor(0.0, device=DEVICE)
    return loss

# ### TODO ###: add metric computations (mAP, IoU, etc.)
def compute_metrics(outputs, targets):
    metrics = {}
    return metrics


## Optimization

In [ ]:
# ### TODO ###: configure optimizer, LR scheduler, and training hyperparameters
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-4
NUM_EPOCHS = 10

optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
scheduler = None  # ### TODO ###: optionally add LR scheduler


## Training Loop

In [ ]:
def train_one_epoch(model, dataloader, optimizer):
    model.train()
    running_loss = 0.0

    for step, (images, targets) in enumerate(dataloader):
        # ### TODO ###: move images/targets to DEVICE and preprocess as needed
        outputs = model(images, targets)
        loss = compute_loss(outputs, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        # ### TODO ###: add logging / visualization hooks
    return running_loss / max(1, len(dataloader))

@torch.no_grad()
def evaluate(model, dataloader):
    model.eval()
    eval_loss = 0.0
    aggregated_metrics = {}

    for images, targets in dataloader:
        outputs = model(images, targets)
        loss = compute_loss(outputs, targets)
        eval_loss += loss.item()
        metrics = compute_metrics(outputs, targets)
        # ### TODO ###: accumulate metrics (mean IoU, mAP, etc.)

    eval_loss /= max(1, len(dataloader))
    # ### TODO ###: finalize aggregated metrics
    return eval_loss, aggregated_metrics


In [ ]:
def save_checkpoint(model, optimizer, epoch, best_metric=None):
    checkpoint = {
        "model_state": model.state_dict(),
        "optimizer_state": optimizer.state_dict(),
        "epoch": epoch,
        "best_metric": best_metric,
    }
    ckpt_path = OUTPUT_DIR / f"checkpoint_epoch_{epoch:03d}.pth"
    torch.save(checkpoint, ckpt_path)
    return ckpt_path

def load_checkpoint(model, optimizer, checkpoint_path: Path):
    state = torch.load(checkpoint_path, map_location=DEVICE)
    model.load_state_dict(state["model_state"])
    optimizer.load_state_dict(state["optimizer_state"])
    return state


In [ ]:
# ### TODO ###: configure early stopping / model selection criteria
best_val_metric = None
history = {"train_loss": [], "val_loss": [], "val_metrics": []}

for epoch in range(1, NUM_EPOCHS + 1):
    train_loss = train_one_epoch(model, train_loader, optimizer)
    val_loss, val_metrics = evaluate(model, val_loader)

    history["train_loss"].append(train_loss)
    history["val_loss"].append(val_loss)
    history["val_metrics"].append(val_metrics)

    print(f"Epoch {epoch}: train_loss={train_loss:.4f}, val_loss={val_loss:.4f}")
    # ### TODO ###: print or log validation metrics

    if scheduler is not None:
        scheduler.step()

    # ### TODO ###: implement best model tracking based on desired metric
    save_checkpoint(model, optimizer, epoch)

# ### TODO ###: persist training history (e.g., to JSON/CSV)


## Evaluation & Visualization

In [ ]:
# ### TODO ###: generate qualitative visualizations for predictions
def visualize_predictions(images, targets, outputs):
    pass

# ### TODO ###: load best checkpoint before running final evaluation
# state = load_checkpoint(model, optimizer, OUTPUT_DIR / "best.pth")
# val_loss, val_metrics = evaluate(model, val_loader)
# print(val_metrics)


## Inference

In [ ]:
@torch.no_grad()
def run_inference(model, image):
    model.eval()
    # ### TODO ###: preprocess single image and run forward pass
    outputs = model(image)
    # ### TODO ###: postprocess outputs into bounding boxes / scores
    return outputs

# ### TODO ###: add inference-on-directory or video helpers if needed


## Next Steps

- Replace each `### TODO ###` with assignment-specific code for data parsing, model architecture, and evaluation.
- Add experiment tracking (TensorBoard, Weights & Biases, etc.) if desired.
- Integrate assignment-specific metrics and submission formatting.